In [ ]:
# importing all the neccesary libraries
import pandas as pd
from pandasql import sqldf
from plotly.offline import plot
import plotly.graph_objs as go

In [ ]:
data = pd.read_excel ("hb.xlsx")
pysqldf = lambda q: sqldf(q, globals()) # a function to help use sql querries

In [ ]:
# new data is created from the big data to the average wages of employees froma all the satats
# receiving wages grater than $60 000
newdata = pysqldf("select round(avg(prevailing_wage), 2) as apw, worksite_state as site_state from data where pw_source_year=2016 group by worksite_state having avg(prevailing_wage) > 60000;")

In [ ]:
# a bar graph to show the average prevailing wage in all state greater than $60,000
fig = go.Figure([go.Bar(
    x = newdata['site_state'], 
    y = newdata['apw'])])
  
#data=go.Data([trace]) 
#layout=go.Layout(title="average prevailing wage per state in 2016",
  #xaxis={'title':'worksite_state'},  
 # yaxis={'title':'average prevailing wage'})  

figure=go.Figure(data=data,layout=layout)
plot(figure)

In [ ]:
# a scatter bubble map to show employees of the various states receiving salary greater than $60 000
fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        locations = newdata['site_state'],
        marker = dict(
            size = newdata['apw']/3000,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width = 1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = newdata['apw'],
            colorbar_title="Wages > $60000"
),
        text = newdata['apw']
))

fig.update_layout(title = "Employees receiving greater than $60000 in various states in 2016",
                   geo = dict(
                       scope='usa', 
                       projection_type='albers usa',
                       showcountries = False,
                       landcolor = "rgb(150, 220, 250)",
                       #subunitcolor = "rgb(217, 217, 217)",
                       #countrycolor = "rgb(100, 217, 217)",
                       #countrywidth = 0.5,
                       #subunitwidth = 0.5
                        ),
                  )
plot(fig)

In [ ]:
# a choropleth to show employees of the various states receiving salary greater than $60 000
data = dict(type='choropleth',
            locations=newdata['site_state'],
            locationmode='USA-states',
            text=newdata['site_state'],
            z=newdata['apw'],
            colorscale='Reds',
            autocolorscale=False,
            #text = newdata['apw'], # hover text
            marker_line_color='white', # line markers between states
            colorbar_title="THOUSANDS USD"
            ) 

layout = dict(
    title="states paying more than $60,000 in 2016",
    geo = dict(scope='usa',
               showlakes= True,
               projection=go.layout.geo.Projection(type = 'albers usa'),
               lakecolor='rgb(150, 200, 255)')
              )

figure= go.Figure(data=[data], layout=layout)
plot(figure)


In [ ]:
## selecting data for average petaining wage, willful_violators, full_time position, h1bdependent and agents
# for those who are certified in 2016 in the state of new york
newdata6 = pysqldf("select avg(prevailing_wage) AS apw, COUNT(willful_violator) AS will, 
                   COUNT(full_time_position) AS ft, 
                   COUNT(h1b_dependent) AS h1b, 
                   COUNT(agent_representing_employer) AS agent, 
                   CASE_STATUS,
                   WORKSITE_STATE
                   FROM data 
                   WHERE (CASE_STATUS == 'CERTIFIED')
                   AND (pw_source_year=='2016')
                   AND (WORKSITE_STATE=='NY')
                   GROUP BY case_status;"))

newdata6.head()

In [ ]:
# selecting data for average petaining wage, willful_violators, full_time position, h1bdependent and agents
# for those who are denied in 2016 in the state of carlifonia

q = """SELECT round(avg(prevailing_wage), 2) AS apw, 
COUNT(willful_violator) AS will, 
COUNT(full_time_position) AS ft, 
COUNT(h1b_dependent) AS h1b, 
COUNT(agent_representing_employer) AS agent, 
CASE_STATUS,
WORKSITE_STATE
FROM data 
WHERE (CASE_STATUS == 'DENIED')
AND (pw_source_year=='2016')
AND (WORKSITE_STATE=='CA')
GROUP BY case_status;

"""
newdata5 = pysqldf(q)

newdata5.head()

In [ ]:
# selecting data for average petaining wage, willful_violators, full_time position, h1bdependent and agents
# for those who are denied in 2016 in the state of new york
select = """SELECT round(avg(prevailing_wage), 2) AS apw, 
COUNT(willful_violator) AS will, 
COUNT(full_time_position) AS ft, 
COUNT(h1b_dependent) AS h1b, 
COUNT(agent_representing_employer) AS agent, 
CASE_STATUS,
WORKSITE_STATE
FROM data 
WHERE (CASE_STATUS == 'DENIED')
AND (pw_source_year=='2016')
AND (WORKSITE_STATE=='NY')
GROUP BY case_status

"""
newdata7 = pysqldf(select)

newdata7

In [ ]:
# a stacked bar chart to show either certified or denied based on the variables from the select statement above
trace1 = go.Bar(
  x = newdata4['CASE_STATUS'], 
  y = newdata4['will']
)


trace2 = go.Bar(
 x =newdata1 ['CASE_STATUS'], 
 y = newdata6['agent']
)
 
trace3= go.Bar(
 x=newdata5['CASE_STATUS'],
 y= newdata5['h1b']
)

trace4=go.Bar(
    x=newdata3['CASE_STATUS'],
    y=newdata7['ft'])

data=([trace1, trace2, trace3, trace4]) 
layout=go.Layout(title="average prevailing wage per state in 2016",
  #xaxis={'title':''}, 
                 barmode='stack')
  #yaxis={'title':''})  

figure=go.Figure(data=data,layout=layout)
plot(figure)
